In [1]:
import chipwhisperer as cw
import time 
import os
import sys
import struct

In [2]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'
CRYPTO_TARGET = "NONE"
BIN = 'main-CWLITEARM.hex'

PATH = '/home/tjaros/Documents/thesis/csidh-target/src/'

In [3]:
def reset_target(scope):
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    time.sleep(0.05)

In [4]:
# Setup our capture and target boards.
########################################
scope = cw.scope()
scope.default_setup()
target = cw.target(scope, cw.targets.SimpleSerial2, flush_on_err=False)
########################################
prog = cw.programmers.STM32FProgrammer

time.sleep(0.1)
scope.default_setup()

In [5]:
os.chdir(PATH)
os.system("make clean PLATFORM={PLATFORM} CRYPTO_TARGET={CRYPTO_TARGET} SS_VER={SS_VER} FUNC_SEL=TRACE")
os.system(f"make PLATFORM={PLATFORM} CRYPTO_TARGET={CRYPTO_TARGET} SS_VER={SS_VER} FUNC_SEL=TRACE")

simpleserial/Makefile.simpleserial:24: *** Invalid SimpleSerial version: {SS_VER}; allowed verions: 
  +---------+--------------+
  | Version | SS_VER value |
  +---------+--------------+
  | V1.0    | SS_VER_1_0   |
  | V1.1    | SS_VER_1_1   |
  | V2.1    | SS_VER_2_1   |
  +---------+--------------+
.  Stop.


SS_VER set to SS_VER_2_1
rm -f -- main-CWLITEARM.hex
rm -f -- main-CWLITEARM.eep
rm -f -- main-CWLITEARM.cof
rm -f -- main-CWLITEARM.elf
rm -f -- main-CWLITEARM.map
rm -f -- main-CWLITEARM.sym
rm -f -- main-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- main.s csidh.s fp.s mont.s randombytes.s uint.s parametrization.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- main.d csidh.d fp.d mont.d randombytes.d uint.d parametrization.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- main.i csidh.i fp.i mont.i randombytes.i uint.i parametrization.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i
.
-------- begin --------
arm-none-eabi-gcc (Arm GNU Toolchain 12.2.Rel1 (Build arm-12.24)) 12.2.1 20221205
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPO

/nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/bin/ld: /nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-closer.o): in function `_close_r':
closer.c:(.text._close_r+0xc): warning: _close is not implemented and will always fail
/nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/bin/ld: /nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-eabi/lib/thumb/v7e-m/nofp/libg_nano.a(libc_a-fstatr.o): in function `_fstat_r':
fstatr.c:(.text._fstat_r+0xe): warning: _fstat is not implemented and will always fail
/nix/store/8w6rddmpg7b1xg2sysjq7kz0aqfpvw12-gcc-arm-embedded-12.2.rel1/bin/../lib/gcc/arm-none-eabi/12.2.1/../../../../arm-none-

0

In [6]:
scope.default_setup()
scope.clock.adc_src = 'clkgen_x1'

FIRMWARE_PATH = PATH + BIN

FIRMWARE_PATH

'/home/tjaros/Documents/thesis/csidh-target/src/main-CWLITEARM.hex'

In [7]:
reset_target(scope)
target.flush()

In [8]:
cw.program_target(scope, prog, FIRMWARE_PATH)
if SS_VER=='SS_VER_2_1':
    target.reset_comms()

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 15195 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 15195 bytes


In [9]:
target.get_simpleserial_commands()

[{'cmd': b'v', 'len': 0, 'flags': 0},
 {'cmd': b'w', 'len': 0, 'flags': 0},
 {'cmd': b'1', 'len': 0, 'flags': 0},
 {'cmd': b'2', 'len': 0, 'flags': 0},
 {'cmd': b'3', 'len': 0, 'flags': 0},
 {'cmd': b'4', 'len': 0, 'flags': 0},
 {'cmd': b'5', 'len': 0, 'flags': 0}]

## Read public

In [10]:
target.flush()
target.send_cmd('2', 0, bytearray([]))
print(target.simpleserial_read())

CWbytearray(b'00 00 00 00 00 00 00 00')


## Read private

In [11]:
target.flush()
target.send_cmd('4', 0, bytearray([]))
print(target.simpleserial_read())

CWbytearray(b'00 00 00')


## Set public
if `scmd` == 0x01 we set it to base curve (A = 0)

In [12]:
target.send_cmd('1', 0x01, bytearray([0x00]*8))

## Perform some isogenies

In [13]:
# reset
reset_target(scope)
target.flush()


# set public to A=0
target.send_cmd('1', 0x01, bytearray([0x00]*8))
# read public
time.sleep(0.1)
target.flush()
target.send_cmd('2', 0, bytearray([]))
print('Public: ',target.simpleserial_read())


# set secret to [4, -6, 8] because the algorithm computes for inverted key
target.flush()
target.send_cmd('3', 0x00, struct.pack('b', -4) + struct.pack('b', 6) + struct.pack('b', -8))
time.sleep(0.1)
# read secret
target.flush()
target.send_cmd('4', 0, bytearray([]))
print('Private:', target.simpleserial_read())

# run the alg
target.flush()
target.send_cmd('5', 0x00, bytearray([]))

# read public
time.sleep(0.4)
target.flush()
target.send_cmd('2', 0, bytearray([]))
print('Public: ',target.simpleserial_read())



Public:  CWbytearray(b'00 00 00 00 00 00 00 00')
Private: CWbytearray(b'fc 06 f8')
Public:  CWbytearray(b'0d 01 00 00 00 00 00 00')


In [14]:
# read public
time.sleep(0.5)
target.flush()
target.send_cmd('2', 0, bytearray([]))
print('Public: ',target.simpleserial_read())


# set secret to [-4, 6, -8]
target.flush()
target.send_cmd('3', 0x00, struct.pack('b', 4) + struct.pack('b', -6) + struct.pack('b', 8))
time.sleep(0.5)
# read secret
target.flush()
target.send_cmd('4', 0, bytearray([]))
print('Private:', target.simpleserial_read())

# run the alg
target.flush()
target.send_cmd('5', 0x00, bytearray([]))

# read public
time.sleep(0.5)
target.flush()
target.send_cmd('2', 0, bytearray([]))
print('Public: ',target.simpleserial_read())



Public:  CWbytearray(b'0d 01 00 00 00 00 00 00')
Private: CWbytearray(b'04 fa 08')
Public:  CWbytearray(b'00 00 00 00 00 00 00 00')


In [15]:
reset_target(scope)
target.flush()

In [16]:
scope.dis()
target.dis()